# TCGA-BRCA Data Loading

This notebook loads TCGA-BRCA RNA-seq and clinical data from the GDC API.

In [5]:
import sys
from pathlib import Path
import pandas as pd

# Add src to path
sys.path.insert(0, str(Path().resolve().parent / "src"))

from data_loader import TCGADataLoader

## Initialize Data Loader

In [6]:
# Initialize loader
loader = TCGADataLoader(data_dir="../data")

## Load File Manifest

In [7]:
# Get file manifest for RNA-seq expression data
manifest = loader.get_file_manifest()
print(f"Found {len(manifest)} RNA-seq files")
manifest.head()

Querying GDC API for Gene Expression Quantification files...
Found 1231 files
Saved manifest to ..\data\file_manifest.tsv
Found 1231 RNA-seq files


,file_id,file_name,file_size,case_id,sample_id
0,744a6d3d-b666-49aa-8d26-47f34e3d1eb5,94027f46-390c-4dda-ab89-cb1ac0a291cd.rna_seq.a...,4231952,878f975b-94fd-4d69-b7e7-1ed3ac2ee438,9321161b-d568-4d38-aed3-956786a06329
1,4ecc1f1a-8ff4-4552-a5e8-7a9652b6d1d5,df45fb41-4511-4dab-b865-fcdfcda0400a.rna_seq.a...,4238832,e4fc0909-f284-4471-866d-d8967b6adcbc,58bfe278-a80f-4722-b286-4d966214d244
2,1ace2a0c-773d-45b5-8fd6-968c88731bbb,c33ecc28-d5ba-4416-b93d-445b7883b6e8.rna_seq.a...,4239626,e5aae05a-478e-4a55-a27c-12b2b4be302a,1d8f72b7-8596-40e8-afdf-244dd897758c
3,2d5b0962-b5c4-4482-9f28-47e4dcdb6df6,602e3d24-a45d-4fe4-9b70-a679d93f470d.rna_seq.a...,4221019,16fc3677-0393-4ed1-ad3f-c8355f056369,a31dec9b-3626-4cb0-8265-1425d5ce1c1e
4,84cff855-d17a-4c09-b432-53893ae69c1e,03d891b3-8faf-4384-94ce-2015f1ca5df0.rna_seq.a...,4217372,33fe5833-d7a0-43d7-a03d-56985ea448a6,2a679581-38ad-4742-8d7b-89015df11e7d


## Load Clinical Data

In [8]:
# Get clinical data
clinical = loader.get_clinical_data()
print(f"Found {len(clinical)} cases")
clinical.head()

Querying GDC API for clinical data...
Found 1098 cases
Saved clinical data to ..\data\clinical_data.tsv
Found 1098 cases


,case_id,age_at_index,gender,race,vital_status,days_to_death,days_to_birth,primary_diagnosis,tumor_stage,ajcc_pathologic_t,ajcc_pathologic_n,ajcc_pathologic_m
0,e3935ce4-64d3-4a66-ba11-d308b844b410,None,None,None,None,None,None,None,None,None,None,None
1,e3b555aa-7f0a-49c6-9b13-182c61a144c1,None,None,None,None,None,None,None,None,None,None,None
2,17ca61a2-607a-45ff-88fa-ef72e80bf891,None,None,None,None,None,None,None,None,None,None,None
3,7d681cc6-689d-41c8-9e84-e13733089ec9,None,None,None,None,None,None,None,None,None,None,None
4,17f275c1-a0d4-487d-8f02-ea279584b4cd,None,None,None,None,None,None,None,None,None,None,None


## Load PAM50 Subtypes

In [10]:
# Get PAM50 subtype annotations (with auto-download if not found)
pam50 = loader.get_pam50_subtypes(auto_download=True)
if not pam50.empty:
    print(f"Found {len(pam50)} PAM50 annotations")
    print(f"\nPAM50 Subtype Distribution:")
    if 'pam50_subtype' in pam50.columns:
        print(pam50['pam50_subtype'].value_counts())
    pam50.head()
else:
    print("PAM50 subtypes not found. Trying manual download...")
    try:
        file_path = loader.download_pam50_from_url()
        pam50 = loader.load_pam50_from_file(str(file_path))
        print(f"Successfully loaded {len(pam50)} PAM50 annotations")
    except Exception as e:
        print(f"Failed to download PAM50: {e}")

Querying GDC API for PAM50 subtype annotations...
  Method 1: Checking annotations endpoint...
  Method 2: Searching for supplementary files with PAM50...
  Method 3: Checking clinical data for subtype fields...

  Method 4: Attempting automatic download from TCGA publications...
PAM50 file already exists: ..\data\BRCA.547.PAM50.SigClust.Subtypes.txt
Loading PAM50 subtypes from file: ..\data\BRCA.547.PAM50.SigClust.Subtypes.txt
  Loaded 547 rows from TSV file
  Found 547 samples with PAM50 subtypes
Found 547 PAM50 annotations

PAM50 Subtype Distribution:
pam50_subtype
LumA      232
LumB      129
Basal      98
Her2       58
Normal     30
Name: count, dtype: int64


## Data Summary

In [ ]:
# Summary statistics
print("=" * 60)
print("DATA SUMMARY")
print("=" * 60)

print(f"\nFile Manifest: {len(manifest)} files")
print(f"Clinical Data: {len(clinical)} cases")
if not pam50.empty:
    print(f"PAM50 Annotations: {len(pam50)} cases")

print("\nClinical Data Columns:")
for col in clinical.columns:
    print(f"  - {col}")

## Data Cleaning & Preparation

In [ ]:
# Check data types and basic info
print("Clinical Data Info:")
print(clinical.info())
print("\n" + "="*60)
print("\nManifest Info:")
print(manifest.info())
print("\n" + "="*60)
if not pam50.empty:
    print("\nPAM50 Info:")
    print(pam50.info())

In [ ]:
# Check for missing values across all datasets
print("Missing Values in Clinical Data:")
missing_clinical = clinical.isnull().sum()
missing_clinical = missing_clinical[missing_clinical > 0].sort_values(ascending=False)
if len(missing_clinical) > 0:
    print(missing_clinical)
    print(f"\nTotal missing values: {missing_clinical.sum()}")
    print(f"Percentage missing: {(missing_clinical.sum() / clinical.size) * 100:.2f}%")
else:
    print("No missing values found!")

print("\n" + "="*60)
print("Missing Values in Manifest:")
missing_manifest = manifest.isnull().sum()
missing_manifest = missing_manifest[missing_manifest > 0].sort_values(ascending=False)
if len(missing_manifest) > 0:
    print(missing_manifest)
else:
    print("No missing values found!")

print("\n" + "="*60)
if not pam50.empty:
    print("Missing Values in PAM50:")
    missing_pam50 = pam50.isnull().sum()
    missing_pam50 = missing_pam50[missing_pam50 > 0].sort_values(ascending=False)
    if len(missing_pam50) > 0:
        print(missing_pam50)
    else:
        print("No missing values found!")

In [ ]:
# Clean clinical data
clinical_clean = clinical.copy()

# Convert numeric columns
numeric_cols = ['age_at_index', 'days_to_death', 'days_to_birth']
for col in numeric_cols:
    if col in clinical_clean.columns:
        clinical_clean[col] = pd.to_numeric(clinical_clean[col], errors='coerce')

# Clean up tumor stage (remove leading text like 'stage ')
if 'tumor_stage' in clinical_clean.columns:
    clinical_clean['tumor_stage'] = clinical_clean['tumor_stage'].str.replace('stage ', '', case=False, regex=False)

# Standardize gender values
if 'gender' in clinical_clean.columns:
    clinical_clean['gender'] = clinical_clean['gender'].str.strip().str.title()

# Create age groups
if 'age_at_index' in clinical_clean.columns:
    clinical_clean['age_group'] = pd.cut(
        clinical_clean['age_at_index'],
        bins=[0, 40, 50, 60, 70, 100],
        labels=['<40', '40-50', '50-60', '60-70', '70+']
    )

print("Clinical data cleaned!")
print(f"Original shape: {clinical.shape}")
print(f"Cleaned shape: {clinical_clean.shape}")
print(f"\nFirst few rows:")
clinical_clean.head()

## Clean PAM50 Subtypes

In [ ]:
# Clean PAM50 subtypes if available
if not pam50.empty:
    pam50_clean = pam50.copy()
    
    # Standardize PAM50 subtype names (remove extra spaces, standardize case)
    if 'pam50_subtype' in pam50_clean.columns:
        pam50_clean['pam50_subtype'] = pam50_clean['pam50_subtype'].str.strip()
        # Common PAM50 subtypes: LumA, LumB, Basal, Her2, Normal
        # Map variations to standard names
        subtype_mapping = {
            'Luminal A': 'LumA',
            'Luminal B': 'LumB',
            'Basal-like': 'Basal',
            'HER2-enriched': 'Her2',
            'HER2': 'Her2',
            'Normal-like': 'Normal'
        }
        pam50_clean['pam50_subtype'] = pam50_clean['pam50_subtype'].replace(subtype_mapping)
        
        # Filter to only valid subtypes (drop Unknown/NA)
        valid_subtypes = ['LumA', 'LumB', 'Basal', 'Her2', 'Normal']
        pam50_clean = pam50_clean[pam50_clean['pam50_subtype'].isin(valid_subtypes)].copy()
        
        print(f"PAM50 subtypes cleaned!")
        print(f"Original count: {len(pam50)}")
        print(f"Cleaned count: {len(pam50_clean)}")
        print(f"\nSubtype distribution:")
        print(pam50_clean['pam50_subtype'].value_counts())
        pam50_clean.head()
    else:
        print("No 'pam50_subtype' column found in PAM50 data")
        pam50_clean = pam50.copy()
else:
    print("No PAM50 data available")
    pam50_clean = pd.DataFrame()

## Merge Clinical Data with PAM50 Subtypes

In [ ]:
# Merge clinical data with PAM50 subtypes
if not pam50_clean.empty:
    # Merge on case_id
    clinical_with_subtypes = clinical_clean.merge(
        pam50_clean[['case_id', 'pam50_subtype']],
        on='case_id',
        how='left'
    )
    
    print(f"Merged dataset shape: {clinical_with_subtypes.shape}")
    print(f"\nCases with PAM50 subtypes: {clinical_with_subtypes['pam50_subtype'].notna().sum()}")
    print(f"Cases without PAM50 subtypes: {clinical_with_subtypes['pam50_subtype'].isna().sum()}")
    print(f"\nSubtype distribution:")
    print(clinical_with_subtypes['pam50_subtype'].value_counts(dropna=False))
    
    clinical_with_subtypes.head()
else:
    print("Cannot merge: PAM50 data not available")
    clinical_with_subtypes = clinical_clean.copy()

## Explore Cleaned Data

In [ ]:
# Summary statistics for cleaned clinical data
if not clinical_with_subtypes.empty:
    print("Summary Statistics:")
    print("="*60)
    
    # Categorical variables
    if 'gender' in clinical_with_subtypes.columns:
        print("\nGender Distribution:")
        print(clinical_with_subtypes['gender'].value_counts(dropna=False))
    
    if 'vital_status' in clinical_with_subtypes.columns:
        print("\nVital Status Distribution:")
        print(clinical_with_subtypes['vital_status'].value_counts(dropna=False))
    
    if 'tumor_stage' in clinical_with_subtypes.columns:
        print("\nTumor Stage Distribution:")
        print(clinical_with_subtypes['tumor_stage'].value_counts(dropna=False).head(10))
    
    if 'age_group' in clinical_with_subtypes.columns:
        print("\nAge Group Distribution:")
        print(clinical_with_subtypes['age_group'].value_counts(dropna=False))
    
    # Numeric variables
    print("\nNumeric Statistics:")
    numeric_cols_clean = clinical_with_subtypes.select_dtypes(include=['float64', 'int64']).columns
    if len(numeric_cols_clean) > 0:
        print(clinical_with_subtypes[numeric_cols_clean].describe())

## Save Cleaned Data

In [ ]:
# Save cleaned datasets
data_dir = Path("../data")

# Save cleaned clinical data
clinical_clean_path = data_dir / "clinical_data_clean.tsv"
clinical_clean.to_csv(clinical_clean_path, sep="\t", index=False)
print(f"Saved cleaned clinical data to: {clinical_clean_path}")

# Save clinical data with subtypes if available
if not pam50_clean.empty:
    clinical_subtypes_path = data_dir / "clinical_data_with_subtypes.tsv"
    clinical_with_subtypes.to_csv(clinical_subtypes_path, sep="\t", index=False)
    print(f"Saved clinical data with subtypes to: {clinical_subtypes_path}")
    
    # Save cleaned PAM50
    pam50_clean_path = data_dir / "pam50_subtypes_clean.tsv"
    pam50_clean.to_csv(pam50_clean_path, sep="\t", index=False)
    print(f"Saved cleaned PAM50 subtypes to: {pam50_clean_path}")

print("\n" + "="*60)
print("Data cleaning complete!")
print("="*60)

## Next Steps

1. ✅ **Data loaded**: Manifest, Clinical, and PAM50 subtypes
2. ✅ **Data cleaned**: Missing values, data types, standardization
3. ✅ **Data merged**: Clinical data with PAM50 subtypes
4. ⏭️ **Next**: Download and process expression data
   - Download expression files using `loader.download_file(file_id, file_name)`
   - Parse expression data (TSV format)
   - Merge expression data with clinical annotations
   - Filter for samples with PAM50 subtypes
   - Proceed to preprocessing and ML pipeline

## Clean Clinical Data